#1 자연어 처리 학습시키기

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
# 모델 불러오기(학습데이터: 네이버 영화 평가)
from keras.models import load_model
model = load_model('movie_review_model.h5')

In [4]:
def tokenize(doc):
    #형태소와 품사를 join
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [5]:
import pandas as pd
train_df = pd.read_csv("ratings_train.txt", "\t")

In [6]:
train_df.isnull().any() #document에 null값이 있다.
train_df['document'] = train_df['document'].fillna(''); #null값을 ''값으로 대체

In [7]:
train_docs = [(tokenize(row[1]), row[2]) for row in train_df.values]

In [8]:
tokens = [t for d in train_docs for t in d[0]]
print("토큰개수:", len(tokens))

토큰개수: 2159921


In [9]:
import nltk
import numpy as np
text = nltk.Text(tokens, name='NMSC')

In [10]:
FREQUENCY_COUNT = 100;
selected_words = [f[0] for f in text.vocab().most_common(FREQUENCY_COUNT)]

In [11]:
#영화평가에서 사용된 단어리스트 문서에서 상위 100개들중 포함되는 단어들의 개수
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [12]:
def predict_sentiment(review):
    token = tokenize(review)
    tfq = term_frequency(token)
    data = np.expand_dims(np.asarray(tfq).astype('float32'), axis=0)
    score = float(model.predict(data))
    #if(score > 0.5):
        #print(f"{review} ==> 긍정 ({round(score*100)}%)")
    #else:
        #print(f"{review} ==> 부정 ({round((1-score)*100)}%)")
    return score

In [13]:
predict_sentiment("재미 정말 없어요")

0.07519051432609558

In [14]:
predict_sentiment("괜찮")

0.5579190850257874

In [15]:
predict_sentiment("카카오 주가 전망 오히려 지금이 매력적인 이유")

0.7493253350257874

#2 분석할 댓글 데이터 가져오기

In [16]:
youtube = pd.read_csv('네이버 댓글.csv')

In [17]:
youtube

,Stock,Date,Video ID,Comment,Comment Like
0,네이버,2020-06-05,Rhf9unTdRGU,"카카오 ㅡ269,000<br>네이버ㅡ239,000<br>일때 매수했는데,<br>지금...",2
1,네이버,2020-06-05,Rhf9unTdRGU,네이버나 카카오는 안사지더라구요<br>살려면 일찍 샀어야 되는데 ㅡㅡ,3
2,네이버,2020-06-05,Rhf9unTdRGU,네이버 카카오 20만원에 사서 늘 빨강빨강하긴한데..한주씩 산게 넘나 가슴아프네요,5
3,네이버,2020-06-05,Rhf9unTdRGU,댓글 처음 써보네요 카카오 네이버 고점에서 물려서 손절해야하나 걱정했는데 영상보고 ...,9
4,네이버,2020-06-05,-6Bw8HDy4oI,쌤.영상보시고 네이버 잡으신분들 축하드립니다.저는 쌤을 안지가 얼마 안되서 너무 안...,1
...,...,...,...,...,...
1642,네이버,2021-07-24,xqKsy-os8fk,카카오 오르면 또 네이버보단 카카오라고 할거임!<br>얼마전까지만해도 네이버보단 카...,4
1643,네이버,2021-07-24,xqKsy-os8fk,네이버가 다왔나<br>또지나 말바꾸기,5
1644,네이버,2021-07-24,xqKsy-os8fk,카카오 네이버 사놓고 강건너 불구경하면서 돈버는 전략도 좋은듯,14
1645,네이버,2021-07-24,xqKsy-os8fk,네이버 오르니 이런 유투브 올라오고 카카오 오르면 카카오 좋다는 영상 올려주고......,31


In [18]:
#1만 조회수 이상인 영상의 댓글들의 감성분석 결과와 점수를 추가
youtube['Sentiment Score'] = 0.0
youtube['Sentiment'] = ""
for i in range(len(youtube)):
    youtube['Sentiment Score'][i] = predict_sentiment(youtube['Comment'][i])
    if youtube['Sentiment Score'][i] > 0.5:
        youtube['Sentiment'][i] = "긍정"
    else:    
        youtube['Sentiment'][i] = "부정"
youtube

,Stock,Date,Video ID,Comment,Comment Like,Sentiment Score,Sentiment
0,네이버,2020-06-05,Rhf9unTdRGU,"카카오 ㅡ269,000<br>네이버ㅡ239,000<br>일때 매수했는데,<br>지금...",2,0.525898,긍정
1,네이버,2020-06-05,Rhf9unTdRGU,네이버나 카카오는 안사지더라구요<br>살려면 일찍 샀어야 되는데 ㅡㅡ,3,0.375683,부정
2,네이버,2020-06-05,Rhf9unTdRGU,네이버 카카오 20만원에 사서 늘 빨강빨강하긴한데..한주씩 산게 넘나 가슴아프네요,5,0.461277,부정
3,네이버,2020-06-05,Rhf9unTdRGU,댓글 처음 써보네요 카카오 네이버 고점에서 물려서 손절해야하나 걱정했는데 영상보고 ...,9,0.470370,부정
4,네이버,2020-06-05,-6Bw8HDy4oI,쌤.영상보시고 네이버 잡으신분들 축하드립니다.저는 쌤을 안지가 얼마 안되서 너무 안...,1,0.263519,부정
...,...,...,...,...,...,...,...
1642,네이버,2021-07-24,xqKsy-os8fk,카카오 오르면 또 네이버보단 카카오라고 할거임!<br>얼마전까지만해도 네이버보단 카...,4,0.301711,부정
1643,네이버,2021-07-24,xqKsy-os8fk,네이버가 다왔나<br>또지나 말바꾸기,5,0.443441,부정
1644,네이버,2021-07-24,xqKsy-os8fk,카카오 네이버 사놓고 강건너 불구경하면서 돈버는 전략도 좋은듯,14,0.605848,긍정
1645,네이버,2021-07-24,xqKsy-os8fk,네이버 오르니 이런 유투브 올라오고 카카오 오르면 카카오 좋다는 영상 올려주고......,31,0.843992,긍정


In [19]:
youtube.to_csv("네이버 감성분석.csv",index=False)